In [4]:
from bs4 import BeautifulSoup as bs
import urllib.request
from nltk import word_tokenize
from nltk.corpus import stopwords
import re
stop = set(stopwords.words('english'))

In [11]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# Github Extract All Links

In [326]:
def extract_githb_page_issues(page_link):
    content = urllib.request.urlopen(page_link)
    data = content.read()
    soup = bs(data,"lxml")
    data = soup.findAll('a', {'class': 'link-gray-dark v-align-middle no-underline h4 js-navigation-open'})
    issues_links = []
    for link in data:
        issues_links.append("https://www.github.com/" + link.get('href'))
    return issues_links

In [25]:
def refine_text(x):
    return [re.sub('[^A-Za-z@]+', ' ', i).lower() for i in x]

In [334]:
# REPO = "facebook/react-native"
REPO = "facebook/watchman"

In [338]:
def extract_repo(REPO):
    all_links = []
    for i in range(100000):
        print("PageNO : ", i)
        url = site + REPO + query_page + str(i) + query_issues
        temp = extract_page_issues(url)
        all_links = all_links + temp
        if len(temp) == 0:
            break
    return list(set(all_links))

In [339]:
git_issues_links = extract_repo(REPO)

In [340]:
git_issues_links

['https://www.github.com//facebook/watchman/issues/649',
 'https://www.github.com//facebook/watchman/issues/648',
 'https://www.github.com//facebook/watchman/issues/647',
 'https://www.github.com//facebook/watchman/issues/646',
 'https://www.github.com//facebook/watchman/issues/644',
 'https://www.github.com//facebook/watchman/issues/643',
 'https://www.github.com//facebook/watchman/issues/642',
 'https://www.github.com//facebook/watchman/issues/641',
 'https://www.github.com//facebook/watchman/issues/640',
 'https://www.github.com//facebook/watchman/issues/638',
 'https://www.github.com//facebook/watchman/issues/633',
 'https://www.github.com//facebook/watchman/issues/631',
 'https://www.github.com//facebook/watchman/issues/630',
 'https://www.github.com//facebook/watchman/issues/629',
 'https://www.github.com//facebook/watchman/issues/628',
 'https://www.github.com//facebook/watchman/issues/627',
 'https://www.github.com//facebook/watchman/issues/626',
 'https://www.github.com//faceb

In [80]:
def extract_snyk_issues(link="https://www.snyk.io/vuln/page/"):
    def extract_issues(link):
        print(link)
        iss = urllib.request.urlopen(link)
        data_1 = iss.read()
        soup_1 = bs(data_1,"lxml")
        data_1 = soup_1.findAll('span',{'class':'l-push-left--sm'})
        issues = []
        for ele in data_1:
            issues.append("https://www.snyk.io/" + ele.find("a").get('href'))
        return issues
    end = "?type=golang"
    issues = []
    for i in range(1000):
        temp = extract_issues(link+str(i+1)+end)
        issues = issues + temp 
        if len(temp) == 0:
            break
    
    git_link=[]
    for i in issues:
        print(i)
        iss = urllib.request.urlopen(i)
        data_1 = iss.read()
        soup_1 = bs(data_1,"lxml")
        data_1 = soup_1.findAll('div',{'class':'card card--markdown'})
        for line in data_1:
            for link in line.findAll("a"):
                temp = link.get("href").replace("/"," ")
                if "issues" in temp or "pull" in temp:
                    git_link.append(link.get("href"))
    return list(set(git_link))

In [312]:
links = extract_snyk_issues()

https://www.snyk.io/vuln/page/1?type=golang
https://www.snyk.io/vuln/page/2?type=golang
https://www.snyk.io/vuln/page/3?type=golang
https://www.snyk.io/vuln/page/4?type=golang
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMGOOGLEGVISORRUNSCBOOTFILTER-72291
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMMOBYMOBY-72364
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMGRAFANAGRAFANA-72290
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMGRAFANAGRAFANAPKGSERVICES-50088
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMGRAFANAGRAFANAPKGAPI-50087
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMHASHICORPPACKERBUILDERAMAZONCOMMON-50085
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMGOOGLEFSCRYPTSECURITY-50084
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMGOOGLEFSCRYPTPAM-50083
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMGOGSGOGSROUTESUSER-50081
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMMINIOMINIOCMD-50080
https://www.snyk.io//vuln/SNYK-GOLANG-GITHUBCOMGOLANGGDDOGOSRC-50079
https://www.snyk.io

In [308]:
def extract_issues(link):
    iss = urllib.request.urlopen(link)
    data_1 = iss.read()
    soup_1 = bs(data_1,"lxml")
    data_1 = soup_1.findAll('td',{'class':'d-block comment-body markdown-body js-comment-body'})
    data_1 = data_1 + soup_1.findAll("a",{"class":"title-link"})
    issues = []
    code = []
    code_list = []
    for i in data_1:
        issues.append(i.get_text())
        code_list = code_list + i.findAll("code")
        code_list = code_list + i.findAll("pre")
    for ele in code_list:
        code.append(ele.get_text())
    for id_x in range(len(issues)):
        for ele in code:
            issues[id_x] = issues[id_x].replace(ele," ",1)
    return issues

In [316]:
doc = []
for i in links:
    doc = doc + extract_issues(i)

In [318]:
doc

['\nFixes #33173.\n#31705 added   when setting up the docker client configuration, but this should also be applied to the daemon.\nIf a file containing CAs for validating clients is provided, only the certs used in that file should be used to validate client connections, and not both the certs in that file and the system root certs.\nIf the union of the system certs and the provided CA certs is desired, the additional CA certs should be added to the system pool, or the system certs added to the provided CA file.\ncc @dmcgowan @thaJeztah\nAlso cc @diogomonica for visibility\n\n',
 '\nLGTM on green\n',
 '\n(sorry misspelled a word in a comment, fixed that :))\n',
 '\nLGTM\n',
 '\nCVE-2018-12608 was assigned to this issue.\n',
 '\n          [17.03] Patch go connections\n          #28\n',
 "\nDescription\nIf   is used, the daemon accepts to many CA's.\nSteps to reproduce the issue:\n\nstart docker daemon with:\n  options\nuse openssl to debug the ssl communication:\n \ncheck output for  :\

In [52]:
# d = extract_issues(issues_links[6])
d = extract_issues(test)

In [53]:
vectorz = TfidfVectorizer(stop_words = stop, 
                          use_idf=True, ngram_range=(1,3))

In [54]:
X = vectorz.fit_transform(refine_text(d))

In [55]:
X[0]

<1x2310 sparse matrix of type '<class 'numpy.float64'>'
	with 510 stored elements in Compressed Sparse Row format>

# LSA

In [57]:
X.shape

(22, 2310)

In [58]:
lsa = TruncatedSVD(n_components=22,n_iter=100)

In [59]:
lsa.fit(X)

TruncatedSVD(algorithm='randomized', n_components=22, n_iter=100,
       random_state=None, tol=0.0)

In [35]:
lsa.components_[0]

array([1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
       1.81834152e-02, 2.42912981e-02, 2.42912981e-02, 2.42912981e-02,
       6.46039187e-02, 6.46039187e-02, 6.46039187e-02, 9.09170758e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
       1.81834152e-02, 5.45502455e-02, 3.63668303e-02, 3.63668303e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 3.63668303e-02,
       1.81834152e-02, 1.81834152e-02, 1.81834152e-02, 1.81834152e-02,
      

In [341]:
terms = vectorz.get_feature_names()
for i, comp in enumerate(lsa.components_):
    termInComp = zip(terms,comp)
    sortedterms = sorted(termInComp,key=lambda x:x[1], reverse=True)[:30]
    print("Concept: ", i)
    for term in sortedterms:
        print(term[0])